In [1]:
import re
import nltk
import pandas as pd
from collections import Counter
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
file = 'receitinhas.csv'
df = pd.read_csv(file, index_col=0)

<h2>Limpeza</h2>

In [3]:
def split_ingredients(ingredients_text):
    ingredients_text = re.sub(r"[\{\}]", "", ingredients_text).strip()
    ingredients_list = [item.strip() for item in ingredients_text.split(',')]
    return ingredients_list

df['Ingredientes'] = df['Ingredientes'].apply(split_ingredients)

In [4]:
df.head(10)

,Receita,Ingredientes
0,Massa de Pizza,"['1/2 kg de farinha de trigo', '1 copo de água..."
1,Receita de Bolo de Cenoura,"['1 pitada de sal', '3 unidades de cenoura méd..."
2,Bolo de Chocolate Molhadinho,"['1 xícara (chá) de leite', '3 unidades de ovo..."
3,Como Fazer Bolinho de Chuva,"['1 colher (sopa) de fermento químico em pó', ..."
4,Bolo de Fubá,"['2 copos de fubá', '1 copo de farinha de trig..."
5,Bolo de Milho,"['1 lata de milho em conserva', '3 unidades de..."
6,Fricassê de Frango,['600 gr de peito de frango cozido e desfiado'...
7,Bolo de Aniversário,"['4 unidades de ovo', '2 xícaras (chá) de açúc..."
8,Bolo de Banana,"['3 unidades de banana nanica', '3 unidades de..."
9,Panquecas,"['1 copo de farinha de trigo', '1 copo de leit..."


Removendo números

In [5]:
def remove_numbers(ingredients):
  return [re.sub(r'\d+\s*', '', item).strip() for item in ingredients]

df['Ingredientes'] = df['Ingredientes'].apply(remove_numbers)

In [6]:
df.head(10)

,Receita,Ingredientes
0,Massa de Pizza,"['/kg de farinha de trigo', 'copo de água', 'g..."
1,Receita de Bolo de Cenoura,"['pitada de sal', 'unidades de cenoura médias ..."
2,Bolo de Chocolate Molhadinho,"['xícara (chá) de leite', 'unidades de ovo', '..."
3,Como Fazer Bolinho de Chuva,"['colher (sopa) de fermento químico em pó', 'x..."
4,Bolo de Fubá,"['copos de fubá', 'copo de farinha de trigo', ..."
5,Bolo de Milho,"['lata de milho em conserva', 'unidades de ovo..."
6,Fricassê de Frango,"['gr de peito de frango cozido e desfiado', 'c..."
7,Bolo de Aniversário,"['unidades de ovo', 'xícaras (chá) de açúcar',..."
8,Bolo de Banana,"['unidades de banana nanica', 'unidades de ovo..."
9,Panquecas,"['copo de farinha de trigo', 'copo de leite', ..."


Todas as letras em minúsculas

In [7]:
def convert_lowercase(ingredients):
    return [item.lower().strip() for item in ingredients]

df['Ingredientes'] = df['Ingredientes'].apply(convert_lowercase)

Remover aspas

In [8]:
def remove_quotes_from_words(ingredient_list):
    cleaned_ingredients = []
    for ingredients in ingredient_list:
        cleaned_ingredients.append([ingredient.strip("'") for ingredient in ingredients])
    return cleaned_ingredients

df['Ingredientes'] = remove_quotes_from_words(df['Ingredientes'])

Remover stopwords

In [9]:
def clean_stopwords(ingredient_list):
    stop_words = set(stopwords.words('portuguese'))
    # print(f"Stopwords carregadas: {stop_words}")
    cleaned_ingredients = []
    for ingredient in ingredient_list:
        ingredient = ingredient.strip()
        words = ingredient.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        cleaned_ingredient = ' '.join(filtered_words)
        cleaned_ingredients.append(cleaned_ingredient)
    return cleaned_ingredients

df['Ingredientes'] = df['Ingredientes'].apply(clean_stopwords)

In [10]:
df.head(10)

,Receita,Ingredientes
0,Massa de Pizza,"[/kg farinha trigo, copo água, gr fermento bio..."
1,Receita de Bolo de Cenoura,"[pitada sal, unidades cenoura médias cruas, xí..."
2,Bolo de Chocolate Molhadinho,"[xícara (chá) leite, unidades ovo, colheres (s..."
3,Como Fazer Bolinho de Chuva,"[colher (sopa) fermento químico pó, xícara (ch..."
4,Bolo de Fubá,"[copos fubá, copo farinha trigo, copo óleo mil..."
5,Bolo de Milho,"[lata milho conserva, unidades ovo, /xícaras (..."
6,Fricassê de Frango,"[gr peito frango cozido desfiado, colheres (so..."
7,Bolo de Aniversário,"[unidades ovo, xícaras (chá) açúcar, /copos fa..."
8,Bolo de Banana,"[unidades banana nanica, unidades ovo, xícaras..."
9,Panquecas,"[copo farinha trigo, copo leite, colher (sopa)..."


Remover medidas e adjetivos

In [11]:
def clean_measurements(ingredients):
    cleaned_ingredients = []
    for item in ingredients:
        item = re.sub(r"\d+|\b(acrescente|american|aproximadamente|batida|batidas|batido|bem|bola|bolas|caixa|caixas|caixinha|caixinhas|chá|cheia|cheias|colher|colheres|copo|copos|cozida|cozidas|cozido|cozidos|crua|cruas|decorar|dente|dentes|derretida|derretidas|derretido|derretidos|desfiado|desfiados|eisenbahn|embebido|envelope|envelopes|fatia|fatias|forte|gosto|grosso|grossa|\bgr\b|grs|grama|gramas|kg|kgs|lactose|lata|latas|litro|litros|maço|maços|magro|média|médias|ml|mls|mililitro|mililitros|morno|orgânico|orgânicos|pacote|pacotes|peneirada|peneiradas|picado|picados|picada|picadas|picadinha|picadinhas|pincelar|pitada|pitadas|polvilhar|quilo|quilos|raspa|raspas|refogar|rodela|rodelas|sobremesa|sopa|sopas|tablete|tabletes|unidade|unidades|xícara|xícaras|zero|esfarelado|especial|fatiada|fresco|grande|grudar|ipa|liga|lipidios|malte|meio|morna|mãos|padrão|pode|pote|preferência|puro|ralada|ralado|recheio|suficiente|tostado|trituradas|triturado|untar|vidro|branca|brancas|branco|brancos|da|de|do|dar|maduro|maduros|madura|maduras|picadinho|picadinhos|picadinha|picadinhas|casca|s|o|gema|gemas|jasmine|mix|gelada|geladas|gelado|gelados)\b", "", item)
        item = item.replace('-', ' ')
        tokens = [word for word in item.split()]
        cleaned_ingredients.append(" ".join(tokens))

    return cleaned_ingredients

df['Ingredientes'] = df['Ingredientes'].apply(clean_measurements)

In [12]:
df.head(10)

,Receita,Ingredientes
0,Massa de Pizza,"[/ farinha trigo, água, fermento biológico, / ..."
1,Receita de Bolo de Cenoura,"[sal, cenoura, () óleo soja, () fermento quími..."
2,Bolo de Chocolate Molhadinho,"[() leite, ovo, () margarina, () açúcar, () ch..."
3,Como Fazer Bolinho de Chuva,"[() fermento químico pó, () farinha trigo, ovo..."
4,Bolo de Fubá,"[fubá, farinha trigo, óleo milho, leite, / açú..."
5,Bolo de Milho,"[milho conserva, ovo, / () fubá, / () açúcar, ..."
6,Fricassê de Frango,"[peito frango, () óleo soja, sal, pimenta rein..."
7,Bolo de Aniversário,"[ovo, () açúcar, / farinha trigo, leite, () ma..."
8,Bolo de Banana,"[banana nanica, ovo, () farinha trigo, / () am..."
9,Panquecas,"[farinha trigo, leite, () amido milho, ovo, sa..."


Remover caracteres não-alfabéticos

In [13]:
def clean_non_alpha(ingredients):
    cleaned_ingredients = []
    for item in ingredients:
        item = re.sub(r'[^a-zA-Zá-úÁ-Ú\s]', '', item)
        item = item.replace('-', ' ')
        tokens = [word for word in item.split()]
        cleaned_ingredients.append(" ".join(tokens))

    return cleaned_ingredients
df['Ingredientes'] = df['Ingredientes'].apply(clean_non_alpha)

In [14]:
df.head(10)

,Receita,Ingredientes
0,Massa de Pizza,"[farinha trigo, água, fermento biológico, óleo..."
1,Receita de Bolo de Cenoura,"[sal, cenoura, óleo soja, fermento químico pó,..."
2,Bolo de Chocolate Molhadinho,"[leite, ovo, margarina, açúcar, chocolate pó, ..."
3,Como Fazer Bolinho de Chuva,"[fermento químico pó, farinha trigo, ovo, leit..."
4,Bolo de Fubá,"[fubá, farinha trigo, óleo milho, leite, açúca..."
5,Bolo de Milho,"[milho conserva, ovo, fubá, açúcar, leite, óle..."
6,Fricassê de Frango,"[peito frango, óleo soja, sal, pimenta reino, ..."
7,Bolo de Aniversário,"[ovo, açúcar, farinha trigo, leite, manteiga, ..."
8,Bolo de Banana,"[banana nanica, ovo, farinha trigo, amido milh..."
9,Panquecas,"[farinha trigo, leite, amido milho, ovo, sal, ..."


<h2>Normalização</h2>

Correção de ortografia - essa biblioteca só fez cagada

In [15]:
# def correct_spelling(ingredients):
#     corrected_words = []
#     corrections = {}  # Dicionário para armazenar correções
#     for ingredient in ingredients:
#         words = ingredient.split()
#         corrected_ingredient = []
#         for word in words:
#             corrected_word = spell.correction(word)
#             if corrected_word != word:
#                 corrections[word] = corrected_word
#             corrected_ingredient.append(corrected_word if corrected_word else word)
#         corrected_words.append(" ".join(corrected_ingredient))
#     print("Palavras corrigidas:", corrections)
#     return corrected_words

# df['Ingredientes'] = df['Ingredientes'].apply(correct_spelling)

Colocar todas as palavras no singular

In [16]:
lemmatizer = WordNetLemmatizer()

def singularize_ingredients(ingredients, recipe_name):
    modified_words = {}
    singularized_ingredients = []

    for ingredient in ingredients:
        words = ingredient.split()
        singularized_ingredient = []

        for word in words:
            singular_word = lemmatizer.lemmatize(word, pos='n')
            if singular_word != word:
                modified_words[word] = singular_word
            singularized_ingredient.append(singular_word)

        singularized_ingredients.append(" ".join(singularized_ingredient))

    if modified_words:
        print(f"Receita: {recipe_name}")
        print("Palavras modificadas:", modified_words)

    return singularized_ingredients

df['Ingredientes'] = df.apply(lambda row: singularize_ingredients(row['Ingredientes'], row['Receita']), axis=1)

# df['Ingredientes'] = df['Ingredientes'].apply(singularize_ingredients)

Receita: Torta de Liquidificador
Palavras modificadas: {'legumes': 'legume'}
Receita: Cuscuz Paulista
Palavras modificadas: {'legumes': 'legume'}
Receita: Pão Integral Caseiro com Aveia e Linhaça
Palavras modificadas: {'germen': 'german'}
Receita: Suflê de Batata com Queijos
Palavras modificadas: {'minas': 'mina'}


In [17]:
df.head(10)

,Receita,Ingredientes
0,Massa de Pizza,"[farinha trigo, água, fermento biológico, óleo..."
1,Receita de Bolo de Cenoura,"[sal, cenoura, óleo soja, fermento químico pó,..."
2,Bolo de Chocolate Molhadinho,"[leite, ovo, margarina, açúcar, chocolate pó, ..."
3,Como Fazer Bolinho de Chuva,"[fermento químico pó, farinha trigo, ovo, leit..."
4,Bolo de Fubá,"[fubá, farinha trigo, óleo milho, leite, açúca..."
5,Bolo de Milho,"[milho conserva, ovo, fubá, açúcar, leite, óle..."
6,Fricassê de Frango,"[peito frango, óleo soja, sal, pimenta reino, ..."
7,Bolo de Aniversário,"[ovo, açúcar, farinha trigo, leite, manteiga, ..."
8,Bolo de Banana,"[banana nanica, ovo, farinha trigo, amido milh..."
9,Panquecas,"[farinha trigo, leite, amido milho, ovo, sal, ..."


In [18]:
# df.to_csv('ingredientes_parciais.csv')

<h2>Todos os ingredientes</h2>

In [19]:
# Está imprimindo cada palavra separadamente
# all_ingredients = set()
# for ingredients_list in df['Ingredientes']:
#     for ingredient in ingredients_list:
#         all_ingredients.update(ingredient.split())

# print("Todos os ingredientes presentes no DataFrame:")
# print(sorted(all_ingredients))

In [20]:
all_ingredients = set()
for ingredients_list in df['Ingredientes']:
    all_ingredients.update(ingredients_list)
print("Todos os ingredientes presentes no DataFrame:")
print(sorted(all_ingredients))

Todos os ingredientes presentes no DataFrame:
['abobrinha', 'abóbora moranga', 'achocolatado pó', 'alecrim', 'alho', 'amendoim', 'amido milho', 'amêndoas', 'aveia', 'azeite', 'azeite extra virgem', 'azeitona verde', 'açúcar', 'açúcar cristal', 'açúcar mascavo', 'banana nanica', 'banana terra verde', 'batata', 'batata doce', 'batata palha', 'bicarbonato sódio', 'biscoito champagne', 'biscoito chocolate baunilha oreo', 'biscoito maisena', 'brócolis', 'cacau pó', 'café', 'café açúcar', 'café cúrcuma pó', 'café essência baunilha', 'café fermento químico pó', 'café orégano', 'café pimenta reino grãos', 'café sal', 'café sal marinho', 'café sementes linhaça', 'calda chocolate', 'caldo galinha', 'caldo legume', 'camarão frango', 'canela china pó', 'canela pau', 'castanha caju', 'cebola', 'cebola desidratada', 'cebolinha verde', 'cenoura', 'cerveja', 'cheiro verde', 'chocolate amargo', 'chocolate granulado', 'chocolate hidrogenado', 'chocolate leite', 'chocolate pó', 'clara ovo neve', 'coco', 